# Imports

In [ ]:
!pip install sentence_transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import textwrap
from scipy import spatial
from sentence_transformers import SentenceTransformer
from pathlib import Path

# Preprocessing

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Twitter/Tweets_climatecrisis_posts_edit.xlsx')
df.head()

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to
0,1784151613017305432,1784151613017305432,2435928960,2024-04-27 09:24:27,"#news \nApril 24-25, 2024\n\nClimate disasters...",NaN
1,1783140655151436107,1783140655151436107,1543556437359484928,2024-04-24 14:27:15,it’s so unfair that the global south is suffer...,NaN
2,1782503903289913750,1782503903289913750,1349149096909668363,2024-04-22 20:17:02,Young Americans know that the climate crisis i...,NaN
3,1783429995852964162,1783429995852964162,551498134,2024-04-25 09:37:00,The most insidious form of climate denial is n...,NaN
4,1782919085145358624,1782919085145358624,1345939602335432706,2024-04-23 23:46:49,AOC is fighting for climate crisis.\nDo you su...,NaN


In [ ]:
df = df.fillna(0)

## Removing short comments
As described in 2.2.2. section, I implement cosine similarity to remove irrelevant comments.

In [ ]:
# Converting all items to strings
df = df.applymap(str)
df.head()

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to
0,1783140655151436107,1783920127097905582,33002078,2024-04-26 18:04:36,@slitheringmilk @iirehsfins Scientifically spe...,1543556437359484928
1,1783140655151436107,1783911935244656743,1024690126402678784,2024-04-26 17:32:03,@slitheringmilk @avispartan1 What will ppl sta...,1543556437359484928
2,1783140655151436107,1783875186460447123,1776068078394327040,2024-04-26 15:06:01,"@slitheringmilk Kamusta, gano ka init ja?",1543556437359484928
3,1783140655151436107,1783858682041946402,1223963128792113152,2024-04-26 14:00:26,@Arse_cloud @slitheringmilk yeah a lot of us c...,1649384374503366657
4,1783140655151436107,1783851196501180786,1407567502977081346,2024-04-26 13:30:42,@Kaiserdotjpg @slitheringmilk It happened to m...,1147162299569860610


In [ ]:
# Removing text shorter than 20 characters

# For comments and tweets
df = df[df['Text'].map(len) > 20]
len(df)

# For posts
df = df[df.apply(lambda x: len(x['selftext']) > 20 or len(x['title']) > 20, axis=1)]
len(df)

1800

### Calculating similarity with transformers

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
df=df.assign(embeddings=df['Text'].apply(lambda x: model.encode(x)))
df.head(3)

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to,embeddings
0,1783140655151436107,1783920127097905582,33002078,2024-04-26 18:04:36,@slitheringmilk @iirehsfins Scientifically spe...,1543556437359484928,"[0.066367574, 0.015628418, 0.13947764, 0.02465..."
1,1783140655151436107,1783911935244656743,1024690126402678784,2024-04-26 17:32:03,@slitheringmilk @avispartan1 What will ppl sta...,1543556437359484928,"[-0.064044505, 0.025292356, 0.08398232, -0.013..."
2,1783140655151436107,1783875186460447123,1776068078394327040,2024-04-26 15:06:01,"@slitheringmilk Kamusta, gano ka init ja?",1543556437359484928,"[-0.049480975, 0.022421906, -0.037423614, 0.02..."


In [ ]:
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [ ]:
# Function prints [number] similar sentences to the input and calculate how similar they are
def print_similar_sentences(input, number):
  input_vector = model.encode(input)

  s=df['embeddings'].apply(lambda x: 1 - spatial.distance.cosine(x, input_vector))
  df['similarity'] = s

  data = df.copy()
  data_sorted = data.sort_values('similarity', ascending=False).head(number)

  for i in range(number):
    print(i+1, wrap(data_sorted['Text'].iloc[i]), np.round(data_sorted['similarity'].iloc[i], decimals = 4))


In [ ]:
# Calculating the similarity of sentences
def calculate_similar_sentences(input):
  input_vector = model.encode(input)

  s=df['embeddings'].apply(lambda x: 1 - spatial.distance.cosine(x, input_vector))
  df['similarity'] = s

In [ ]:
print_similar_sentences('no', 20)

1 @RepJeffries No, it is not 0.325
2 @catturd2 @RepJeffries Yes 0.2586
3 @catturd2 @RepJeffries Yes 0.2586
4 @RepJeffries It's working?  Last year was the hottest on record.
This after you and your ilk have spent hundreds of billions of dollars
to fix it.

Please stop your ... "effort."  You're bankrupting us. 0.2583
5 @RepJeffries Is there a Planet X? 0.258
6 @POTUS Is it though?  Is it? 0.2412
7 @slitheringmilk YES. WE ARE BURNING HERE 0.2406
8 @MontanaSeptic @NoMRNAJabs @OneBadDude_ @POTUS yes it those 0.2396
9 @RodneyClaeys @POTUS @JoeBiden Not very intelligent are you? 0.2373
10 @RepJeffries Have you always been a POS? 0.2315
11 @RepJeffries If you say so… 🤡 0.2242
12 @POTUS Does anyone believe your b.s.? 0.2213
13 @POTUS Teens for Cash 0.2205
14 @RepJeffries Is it?  Tell me how so?  You can just make a statement
without evidence 0.2177
15 @slitheringmilk @avispartan1 What will ppl start doing?  Maybe take
polluting countries to the ICJ? 0.2108
16 @RodneyClaeys @RepJeffries @CAPAc

### Removing process



In [ ]:
to_remove_phrases = ['send','i agree', 'i disagree', 'wish you luck', 'congratulations', 'good luck', 'ok', 'no', 'good job', 'maybe', 'sure', 'ah', 'yeah', 'have a good day', 'thank you']

In [ ]:
for phrase in to_remove_phrases:
  calculate_similar_sentences(phrase)
  df.drop(df[df.similarity > 0.35].index, inplace=True)

In [ ]:
len(df)

1586

In [ ]:
# Checking if it worked
print_similar_sentences('send', 10)

1 @POTUS https://t.co/koI1ULrycd 0.3498
2 @RepJeffries https://t.co/328kaq42oE 0.3498
3 @shadow5bass @Hardtruths100 @POTUS He's a defector that knows more
about it than you do.  https://t.co/WdtOvNDVyK 0.3493
4 @RepJeffries https://t.co/7wvQ364Iz1 0.3491
5 @RepJeffries https://t.co/cnOaqjcJzX 0.3491
6 @TheJonamm @berningman16 @POTUS LOUDER 0.3488
7 @RepJeffries https://t.co/TG8njoqCXs 0.3484
8 @RepJeffries https://t.co/hvgX9kFEe2 0.3484
9 @DanMcGowen3 @Hardtruths100 @POTUS Use your Big Boi words now...  you
can do it. 0.348
10 @RepJeffries https://t.co/836J1u6YOM 0.3476


### Formating the file


In [ ]:
df.head(3)

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to,embeddings,similarity
0,1783140655151436107,1783920127097905582,33002078,2024-04-26 18:04:36,@slitheringmilk @iirehsfins Scientifically spe...,1543556437359484928,"[0.066367574, 0.015628418, 0.13947764, 0.02465...",0.015047
1,1783140655151436107,1783911935244656743,1024690126402678784,2024-04-26 17:32:03,@slitheringmilk @avispartan1 What will ppl sta...,1543556437359484928,"[-0.064044505, 0.025292356, 0.08398232, -0.013...",0.107996
2,1783140655151436107,1783875186460447123,1776068078394327040,2024-04-26 15:06:01,"@slitheringmilk Kamusta, gano ka init ja?",1543556437359484928,"[-0.049480975, 0.022421906, -0.037423614, 0.02...",0.206539


In [ ]:
# Removing two last columns
df.drop(columns = ['embeddings', 'similarity'], inplace=True)

In [ ]:
df.head(3)

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to
0,1783140655151436107,1783920127097905582,33002078,2024-04-26 18:04:36,@slitheringmilk @iirehsfins Scientifically spe...,1543556437359484928
1,1783140655151436107,1783911935244656743,1024690126402678784,2024-04-26 17:32:03,@slitheringmilk @avispartan1 What will ppl sta...,1543556437359484928
2,1783140655151436107,1783875186460447123,1776068078394327040,2024-04-26 15:06:01,"@slitheringmilk Kamusta, gano ka init ja?",1543556437359484928


In [ ]:
# save if comments
comments = df

# save if posts
posts = df

In [ ]:
comments

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to
0,1783140655151436107,1783920127097905582,33002078,2024-04-26 18:04:36,@slitheringmilk @iirehsfins Scientifically spe...,1543556437359484928
1,1783140655151436107,1783911935244656743,1024690126402678784,2024-04-26 17:32:03,@slitheringmilk @avispartan1 What will ppl sta...,1543556437359484928
2,1783140655151436107,1783875186460447123,1776068078394327040,2024-04-26 15:06:01,"@slitheringmilk Kamusta, gano ka init ja?",1543556437359484928
3,1783140655151436107,1783858682041946402,1223963128792113152,2024-04-26 14:00:26,@Arse_cloud @slitheringmilk yeah a lot of us c...,1649384374503366657
4,1783140655151436107,1783851196501180786,1407567502977081346,2024-04-26 13:30:42,@Kaiserdotjpg @slitheringmilk It happened to m...,1147162299569860610
...,...,...,...,...,...,...
1895,1782364547480772815,1782546765452808475,824010578737774592,2024-04-22 23:07:21,@RepJeffries Nearly $7 trillion of public and ...,467823431
1896,1782364547480772815,1782546707479183766,367545145,2024-04-22 23:07:07,"@RepJeffries There is, yall just are keeping i...",467823431
1897,1782364547480772815,1782546649262219328,851166938231435264,2024-04-22 23:06:53,@RepJeffries One world...as China and India ad...,467823431
1898,1782364547480772815,1782546516688650722,824010578737774592,2024-04-22 23:06:22,"@RepJeffries When you have some spare time, pl...",467823431


In [ ]:
# Converting 'Conversation ID', 'Tweet ID', and 'Author ID' to strings in both DataFrames
posts['Conversation ID'] = posts['Conversation ID'].astype(str)
posts['Tweet ID'] = posts['Tweet ID'].astype(str)
posts['Author ID'] = posts['Author ID'].astype(str)
posts['In reply to'] = posts['In reply to'].astype(str)

comments['Conversation ID'] = comments['Conversation ID'].astype(str)
comments['Tweet ID'] = comments['Tweet ID'].astype(str)
comments['Author ID'] = comments['Author ID'].astype(str)
comments['In reply to'] = comments['In reply to'].astype(str)


## Merging posts with comments

In [ ]:
merged_data = []

for index, row in posts.iterrows():
    # Adding the post
    merged_data.append({
        'Conversation ID': row['Conversation ID'],
        'Tweet ID': row['Tweet ID'],
        'Author ID': row['Author ID'],
        'Created At': row['Created At'],
        'Text': row['Text']
    })

    # Finding and adding related comments
    related_comments = comments[comments['Conversation ID'] == row['Conversation ID']]
    for _, comment_row in related_comments.iterrows():
      merged_data.append({
        'Conversation ID': comment_row['Conversation ID'],
        'Tweet ID': comment_row['Tweet ID'],
        'Author ID': comment_row['Author ID'],
        'Created At': comment_row['Created At'],
        'Text': comment_row['Text'],
        'In reply to': comment_row['In reply to']
      })

merged_df = pd.DataFrame(merged_data)

merged_df.head(20)

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to
0,1784151613017305432,1784151613017305432,2435928960,2024-04-27 09:24:27,"#news \nApril 24-25, 2024\n\nClimate disasters...",NaN
1,1783140655151436107,1783140655151436107,1543556437359484928,2024-04-24 14:27:15,it’s so unfair that the global south is suffer...,NaN
2,1783140655151436107,1783920127097905582,33002078,2024-04-26 18:04:36,@slitheringmilk @iirehsfins Scientifically spe...,1543556437359484928
3,1783140655151436107,1783911935244656743,1024690126402678784,2024-04-26 17:32:03,@slitheringmilk @avispartan1 What will ppl sta...,1543556437359484928
4,1783140655151436107,1783875186460447123,1776068078394327040,2024-04-26 15:06:01,"@slitheringmilk Kamusta, gano ka init ja?",1543556437359484928
5,1783140655151436107,1783858682041946402,1223963128792113152,2024-04-26 14:00:26,@Arse_cloud @slitheringmilk yeah a lot of us c...,1649384374503366657
6,1783140655151436107,1783851196501180786,1407567502977081346,2024-04-26 13:30:42,@Kaiserdotjpg @slitheringmilk It happened to m...,1147162299569860610
7,1783140655151436107,1783850770489893167,1407567502977081346,2024-04-26 13:29:00,@OptyrionPrimax @slitheringmilk suffering is p...,1407668348511350788
8,1783140655151436107,1783838260223254773,1759742256565694464,2024-04-26 12:39:18,@slitheringmilk In Argentina we got temperatur...,1543556437359484928
9,1783140655151436107,1783814853976789268,1711509777048895488,2024-04-26 11:06:17,@slitheringmilk https://t.co/BmjnnSFXq8,1543556437359484928


In [ ]:
len(merged_df)

1655

## Adding columns with heuristics and biases

In [ ]:
merged_df['affect_h'] = ''
merged_df['availability_h'] = ''
merged_df['representativeness_h'] = ''
merged_df['confirmation_b']=''
merged_df['all_or_nothing']=''

In [ ]:
merged_df.columns =['Conversation ID', 'Tweet ID', 'Author ID','Created At', 'Text', 'In reply to', 'affect_h', 'availability_h', 'representativeness_h', 'confirmation_b', 'all_or_nothing']

In [ ]:
merged_df.head(3)

,Conversation ID,Tweet ID,Author ID,Created At,Text,In reply to,affect_h,availability_h,representativeness_h,confirmation_b,all_or_nothing
0,1784151613017305432,1784151613017305432,2435928960,2024-04-27 09:24:27,"#news \nApril 24-25, 2024\n\nClimate disasters...",NaN,,,,,
1,1783140655151436107,1783140655151436107,1543556437359484928,2024-04-24 14:27:15,it’s so unfair that the global south is suffer...,NaN,,,,,
2,1783140655151436107,1783920127097905582,33002078,2024-04-26 18:04:36,@slitheringmilk @iirehsfins Scientifically spe...,1543556437359484928,,,,,


In [ ]:
output_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Twitter/climate_crisis_post_and_comments_merged.xlsx'
merged_df.to_excel(output_path, index=False)

# Diving files for 4 annotators
The file division was tricky due to including data for agreement dataset - all annotators shared 30% of the whole dataset

## Creating 4 files

In [ ]:
base_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Diveded by annotator/'
file_names = [f"Reddit_Round2_Annotator_{i}_AI.xlsx" for i in range(1, 5)]  # Names for the 4 files

empty_df = pd.DataFrame()

for file_name in file_names:
    file_path = base_path + file_name
    empty_df.to_excel(file_path, index=False, engine='openpyxl')

print("Empty Excel files have been created.")

## Diving metadata

In [ ]:
base_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Diveded by annotator/'
file_names = [f"Reddit_Round2_Annotator_{i}_AI.xlsx" for i in range(1, 5)]

df1 = pd.read_excel('/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Reddit/Singularity_with_author.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Reddit/AIwars_with_author.xlsx')

# Variables for the number of posts
x = 1092  # Number of posts for the first round
y = 2548   # Number of posts for the subsequent rounds

## TikTok

In [ ]:
# Function to prepend a single quote to the "id" column values, converting them to text format for Excel
def convert_id_to_text_format(df):
    if 'id' in df.columns:
        df['id'] = "'" + df['id'].astype(str)
    return df

# Adjusted read_data function to read 'id' column as string and convert to text format for Excel
def read_data(file_path):
    if Path(file_path).exists():
        df = pd.read_excel(file_path, dtype={'id': str})
        return convert_id_to_text_format(df)
    else:
        return pd.DataFrame()

# Marking the first x entries
if 'First round' not in df.columns:
    df['First round'] = ''
df.loc[:x-1, 'First round'] = '1234'
df = convert_id_to_text_format(df)

current_index = x

# Marking y entries
for i in range(1, 5):
    df.loc[current_index:current_index+y-1, 'First round'] = str(i)
    current_index += y

# Appending the first x entries of df to all specified files
for file_name in file_names:
    file_path = base_path + file_name
    existing_df = read_data(file_path)
    new_data_segment = df.iloc[:x]
    updated_df = pd.concat([existing_df, new_data_segment], ignore_index=True)
    updated_df.to_excel(file_path, index=False, engine='openpyxl')

# Appending subsequent y entries from df across the files, starting again from x
current_index = x
for i, file_name in enumerate(file_names, start=1):
    file_path = base_path + file_name
    existing_df = read_data(file_path)  # Load existing data
    end_index = current_index + y
    new_data_segment = df.iloc[current_index:end_index]  # y entries for each file
    updated_df = pd.concat([existing_df, new_data_segment], ignore_index=True)
    updated_df.to_excel(file_path, index=False, engine='openpyxl')
    current_index = end_index  # Move to the next set of entries

print("Data has been appended to all files.")

Data has been appended to all files.


## Twitter

#### Pilot study

In [ ]:
def convert_ids_to_text_format(df, column_names):
    for column in column_names:
        if column in df.columns:
            df[column] = "'" + df[column].astype(str)
    return df

columns_to_convert = ['Conversation ID', 'Tweet ID', 'Author ID']

df = convert_ids_to_text_format(df, columns_to_convert)

if 'First round' not in df.columns:
    df['First round'] = ''

df.loc[:x-1, 'First round'] = '1234'

def read_data(file_path):
    if Path(file_path).exists():
        df = pd.read_excel(file_path, dtype={col: str for col in columns_to_convert})
        return convert_ids_to_text_format(df, columns_to_convert)
    else:
        return pd.DataFrame()

# Appending the first x entries to all files
for file_name in file_names:
    file_path = base_path + file_name
    existing_df = read_data(file_path)
    new_data_segment = df.iloc[:x]
    updated_df = pd.concat([existing_df, new_data_segment])
    updated_df.to_excel(file_path, index=False, engine='openpyxl')

# Starting index for subsequent entries
current_index = x

# Distribute subsequent y entries across the files
for i, file_name in enumerate(file_names, start=1):
    file_path = base_path + file_name
    existing_df = read_data(file_path)

    end_index = current_index + y
    df.loc[current_index:end_index - 1, 'First round'] = str(i)
    new_data_segment = df.iloc[current_index:end_index]

    updated_df = pd.concat([existing_df, new_data_segment])
    updated_df.to_excel(file_path, index=False, engine='openpyxl')

    current_index = end_index

print("Data has been appended to all files.")

Data has been appended to all files.


In [ ]:
def process_and_save_data(df, file_path):

    df['Conversation ID'] = df['Conversation ID'].astype(str)
    df['Second round'] = df['Second round'].astype(str)

    first_two_ids = df[df['Second round'] == '4']['Conversation ID'].drop_duplicates().head(2)

    first_batch = df[df['Conversation ID'].isin(first_two_ids) & (df['Second round'] == '4')]
    second_batch = df[df['Second round'] == '1234']
    remaining_ids = df[~df['Conversation ID'].isin(first_two_ids) & (df['Second round'] == '4')]

    final_df = pd.concat([first_batch, second_batch, remaining_ids])
    final_df.to_excel(file_path, index=False, engine='openpyxl')

In [ ]:
file_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Diveded by annotator/Twitter_Round1_Annotator_4_Climate.xlsx'

In [ ]:
process_and_save_data(df, file_path)

#### Main study


In [ ]:
def convert_ids_to_text_format(df, column_names):
    for column in column_names:
        if column in df.columns:
            # Convert column to string and prepend a single quote
            df[column] = "'" + df[column].astype(str)
    return df

columns_to_convert = ['Conversation ID', 'Tweet ID', 'Author ID', 'In reply to']
df = convert_ids_to_text_format(df, columns_to_convert)

In [ ]:
# Mrking the dataset
def mark_rows(df, x, y, file_path):
    if 'Second round' not in df.columns:
        df['Second round'] = ''

    # Mark the first x rows
    df.loc[:x-1, 'Second round'] = '1234'

    marks = ['1', '2', '3', '4']
    current_index = x
    for mark in marks:
        df.loc[current_index:current_index + y - 1, 'Second round'] = mark
        current_index += y

    df.to_excel(file_path, index=False, engine='openpyxl')

    return df

In [ ]:
file_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Twitter/left_and_new_s2_twitter_climate_merged.xlsx'
mark_rows(df, x, y, file_path)
df

,Conversation ID,Tweet ID,Author ID,Created At,Text,affect_h,availability_h,representativeness_h,confirmation_b,all_or_nothing,First round,In reply to,Second round
0,'1766163071070802308,'1766243130377191891,'108362685,2024-03-08 23:22:32,@PGDynes How about we deal with the mess we. H...,NaN,NaN,NaN,NaN,NaN,NaN,'nan,1234
1,'1766163071070802308,'1766240527526084633,'1351651710947753985,2024-03-08 23:12:11,@PGDynes Really. It was warmer in Roman times.,NaN,NaN,NaN,NaN,NaN,NaN,'nan,1234
2,'1766163071070802308,'1766240385775596014,'1293789256662450176,2024-03-08 23:11:37,@PGDynes Crazy how nature does that. Looks mor...,NaN,NaN,NaN,NaN,NaN,NaN,'nan,1234
3,'1766163071070802308,'1766238518106570857,'1330379976282042368,2024-03-08 23:04:12,@PGDynes Yep and I love it - anyone who is aga...,NaN,NaN,NaN,NaN,NaN,NaN,'nan,1234
4,'1766163071070802308,'1766237097323581534,'1665097084628869122,2024-03-08 22:58:33,@lionsharz @PGDynes fyt,NaN,NaN,NaN,NaN,NaN,NaN,'nan,1234
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,'1782422388451238296,'1782422388451238296,'73754019,2024-04-22 14:53:07,"This Earth Day, I’m grateful we have an admini...",NaN,NaN,NaN,NaN,NaN,NaN,'nan,4
4711,'1784125852801724424,'1784125852801724424,'1780557415769051136,2024-04-27 07:42:05,What does the new â_x0080__x0098_State of the ...,NaN,NaN,NaN,NaN,NaN,NaN,'nan,4
4712,'1783960975877521811,'1783960975877521811,'998825510552723456,2024-04-26 20:46:55,Climate change denialism is part of a class wa...,NaN,NaN,NaN,NaN,NaN,NaN,'nan,
4713,'1782459922166280393,'1782459922166280393,'16451542,2024-04-22 17:22:16,This moment during my recent interview with @J...,NaN,NaN,NaN,NaN,NaN,NaN,'nan,


## Reddit

### Pilot study

In [ ]:
if 'First round' not in df.columns:
    df['First round'] = ''
df.loc[:x-1, 'First round'] = '1234'
current_index = x

for i in range(1, 5):
    df.loc[current_index:current_index+y-1, 'First round'] = str(i)
    current_index += y

for file_name in file_names:
    file_path = base_path + file_name
    existing_df = read_data(file_path)
    new_data_segment = df.iloc[:x]
    updated_df = pd.concat([existing_df, new_data_segment], ignore_index=True)
    updated_df.to_excel(file_path, index=False, engine='openpyxl')

current_index = x
for i, file_name in enumerate(file_names, start=1):
    file_path = base_path + file_name
    existing_df = read_data(file_path)
    end_index = current_index + y
    new_data_segment = df.iloc[current_index:end_index]
    updated_df = pd.concat([existing_df, new_data_segment], ignore_index=True)
    updated_df.to_excel(file_path, index=False, engine='openpyxl')
    current_index = end_index

print("Data has been appended to all files.")

Data has been appended to all files.


### Main study

In [ ]:
def mark_rows_and_save(df1, df2, x, y, file_path1, file_path2):
    if 'Second round' not in df1.columns:
        df1['Second round'] = ''
    if 'Second round' not in df2.columns:
        df2['Second round'] = ''

    # Mark the first x rows in df1 with '1234'
    df1.loc[:x-1, 'Second round'] = '1234'

    # Define marks after the first x as '1' and '2' for every y rows in df1
    marks_df1 = ['1', '2']
    current_index_df1 = x
    for mark in marks_df1:
        df1.loc[current_index_df1:current_index_df1 + y - 1, 'Second round'] = mark
        current_index_df1 += y

    df1.to_excel(file_path1, index=False, engine='openpyxl')

    # Define marks as '3' and '4' repeatedly for every y rows in df2
    marks_df2 = ['3', '4']
    current_index_df2 = 0
    for mark in marks_df2:
        df2.loc[current_index_df2:current_index_df2 + y - 1, 'Second round'] = mark
        current_index_df2 += y

    df2.to_excel(file_path2, index=False, engine='openpyxl')

In [ ]:
file_path1 = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Reddit/Singularity_with_author.xlsx'
file_path2 = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Reddit/AIwars_with_author.xlsx'

mark_rows_and_save(df1, df2, x, y, file_path1, file_path2)

In [ ]:
def process_and_save_data(df1, df2, file_path):

    df1['Second round'] = df1['Second round'].astype(str)
    df2['Second round'] = df2['Second round'].astype(str)

    first_two_posts = df2[df2['Second round'] == '4']['post_id_x'].drop_duplicates().head(2)
    first_batch = df2[df2['post_id_x'].isin(first_two_posts) & (df2['Second round'] == '4')]

    second_batch = df1[df1['Second round'] == '1234']
    remaining_posts = df2[(~df2['post_id_x'].isin(first_two_posts)) & (df2['Second round'] == '4')]

    final_df = pd.concat([first_batch, second_batch, remaining_posts])
    final_df.to_excel(file_path, index=False, engine='openpyxl')

In [ ]:
file_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Diveded by annotator/Reddit_Round2_Annotator_4_AI.xlsx'

In [ ]:
process_and_save_data(df1, df2, file_path)

# Before the main study

## Add column with authors' usernames
After the pilot study, the annotators asked for adding usernames of authors of the comments.

In [ ]:
df1 =  pd.read_excel("/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Reddit/Kopia Singularity_merged.xlsx")
df2 =  pd.read_excel("/content/drive/MyDrive/MA thesis/Data collection/Data for 1st round/Reddit/Comments/Singularity/merged_file_full.xlsx")

result = pd.merge(df1, df2, on='comment_id', how='left')
result

,title,post_id_x,comment_id,text,affect_h,availability_h,representativeness_h,confirmation_b,all_or_nothing,First round,post_id_y,comment_body,comment_author,comment_score,created_utc
0,After building with the OpenAI API,1b0zjt5,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,1b0zjt5,ksf88yl,Thanks for contributing to r/singularity. Howe...,NaN,NaN,NaN,NaN,NaN,NaN,1b0zjt5,Thanks for contributing to r/singularity. Howe...,singularity-ModTeam,1.0,2024-02-27 19:45:37
2,NaN,1b0zjt5,ksc3ugu,"Depends on the application, tbh. If you just w...",NaN,NaN,NaN,NaN,NaN,NaN,1b0zjt5,"Depends on the application, tbh. If you just w...",HalfSecondWoe,14.0,2024-02-27 05:25:55
3,NaN,1b0zjt5,kscea18,why use 3.5 when goody2 is free?,NaN,NaN,NaN,NaN,NaN,NaN,1b0zjt5,why use 3.5 when goody2 is free?,NoNet718,8.0,2024-02-27 07:08:58
4,NaN,1b0zjt5,ksbpwq7,Understanding economics is definitely to the r...,NaN,NaN,NaN,NaN,NaN,NaN,1b0zjt5,Understanding economics is definitely to the r...,sdmat,2.0,2024-02-27 03:37:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19940,"As we approach FDVR with AGI NPCs, gaming is g...",1aivmv6,NaN,If people can talk to realistic characters in ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
19941,SuGaR: Surface-Aligned Gaussian Splatting for ...,1aiti43,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
19942,GPT-4 should be less lazy now - Sama,1air3eu,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
19943,Missing the ASI and Transhumanism,1aiqtwi,NaN,This a philosophical thread about the future ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [ ]:
output_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Reddit/Singularity_with_author.xlsx'
result.to_excel(output_path, index=False)

## Merging data from X
I merged data that left unused from the pilot study with new data for the main study

In [ ]:
paths = [
    '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Twitter/left_s1_twitter_climate_merged.xlsx',
    '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Twitter/new_s2_twitter_climate_merged.xlsx'

]

dataframes = []
for path in paths:
    df = pd.read_excel(path)

    for col in df.columns:
        df[col] = df[col].astype(str)
    dataframes.append(df)


merged_df = pd.concat(dataframes)


for col in merged_df.columns:
    merged_df[col] = merged_df[col].astype(str)


output_path = '/content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Twitter/left_and_new_s2_twitter_climate_merged.xlsx'
merged_df.to_excel(output_path, index=False, engine='openpyxl')

print(f"Merged file saved to {output_path}")

Merged file saved to /content/drive/MyDrive/MA thesis/Data collection/Data for 2nd round/Twitter/left_and_new_s2_twitter_climate_merged.xlsx
